In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: pho
"""
import sys
from threading import Thread
import time # for time.sleep
from ipygany import PolyMesh, Scene, IsoColor, WarpByScalar
import pyvista as pv
import pyvistaqt as pvqt
import colorcet as cc # Colormaps:
import numpy as np
import h5py
import hdf5storage # conda install hdf5storage
from pathlib import Path
import bqplot.scales
import seaborn as sns
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
# import mplcursors
import math # For color map generation
from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv

import ipywidgets as widgets
# from PyQt5 import QtWidgets, uic
from pyvistaqt import QtInteractor, MainWindow
# from pyqt6 import QApplication
from IPython.external.qt_for_kernel import QtGui
from PyQt5.QtWidgets import QApplication

In [5]:
# import PhoPositionalData as pdp
# from PhoPositionalData import load_exported, process_data
from PhoPositionalData.load_exported import *
# from PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
from PhoPositionalData.process_data import *
from PhoPositionalData.plot_data import *
from PhoPositionalData.plotting.animations import * # make_mp4_from_plotter, apply_close_overhead_zoomed_camera_view
from PhoPositionalData.import_data import * # build_spike_positions_list, build_cellID_reverse_lookup_map
from PhoPositionalData.analysis.interactive_placeCell_config import InteractivePlaceCellConfig, VideoOutputModeConfig, PlottingConfig  # VideoOutputModeConfig, InteractivePlaceCellConfigs

In [6]:
# NeuroPy (Diba Lab Python Repo) Loading
try:
    from neuropy import core
except ImportError:
    sys.path.append(r'C:\Users\Pho\repos\NeuroPy') # Windows
    # sys.path.append('/home/pho/repo/BapunAnalysis2021/NeuroPy') # Linux
    # sys.path.append(r'/Users/pho/repo/Python Projects/NeuroPy') # MacOS
    print('neuropy module not found, adding directory to sys.path. \n >> Updated sys.path.')
    from neuropy import core
from neuropy.core.dataSession import DataSessionLoader, DataSession, processDataSession

# basedir = '/media/share/data/Bapun/Day5TwoNovel' # Linux
# basedir = 'R:\data\Bapun\Day5TwoNovel' # Windows
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15' # Windows
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53' # Windows
# basedir = '/Volumes/iNeo/Data/Bapun/Day5TwoNovel' # MacOS

# from neuropy.io.neuroscopeio import NeuroscopeIO

## No Pre-processing required
# base_neuroscope_xml_file = Path(basedir).joinpath('2006-6-08_14-26-15.xml')

# From pre-computed .mat files:
## 07: 
basedir = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'
spike_file = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.spikeII.mat'
neuroscope_xml_file = Path(basedir).joinpath('2006-6-07_11-26-53.xml')

# ## 08:
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15'
# spike_file = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.spikeII.mat' # '2006-6-08_14-26-15.spikeII.mat' # Contains 'spike' flat structure
# neuroscope_xml_file = Path(basedir).joinpath('2006-6-08_14-26-15.xml')

# sess = core.processDataSession(basedir)

In [7]:
curr_args_dict = dict()
curr_args_dict['basepath'] = basedir
curr_args_dict['session_obj'] = DataSession()
# curr_session = DataSessionLoader.default_load_bapun_npy_session_folder(curr_args_dict)
sess = DataSessionLoader.default_load_kamran_flat_spikes_mat_session_folder(curr_args_dict)
sess

	 basepath: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53
	 session_name: 2006-6-07_11-26-53
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.spikeII.mat...
done.
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position_info.mat...
done.
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.epochs_info.mat...
done.
computing linear positions for all active epochs for session...
session.position linear positions could not be computed due to error Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.. Skipping.


DataSession(2006-6-07_11-26-53.xml)

In [8]:
session_name = Path(basedir).parts[-1]
session_position_mat_file_path = Path(basedir).joinpath('{}.position_info.mat'.format(session_name))
position_mat_file = import_mat_file(mat_import_file=session_position_mat_file_path)
# ['microseconds_to_seconds_conversion_factor','samplingRate', 'timestamps', 'x', 'y']
t = position_mat_file['timestamps'].squeeze() # 1, 63192
print('np.shape(t): {}'.format(np.shape(t))) # np.shape(t): (59308,)
x = position_mat_file['x'].squeeze() # 10 x 63192
print('np.shape(x): {}'.format(np.shape(x))) # np.shape(x): (59308,)
y = position_mat_file['y'].squeeze() # 10 x 63192
print('np.shape(y): {}'.format(np.shape(y))) # np.shape(y): (59308,)
position_sampling_rate_Hz = position_mat_file['samplingRate'].item() # In Hz, returns 29.969777
# print('np.shape(position_sampling_rate_Hz): {}'.format(np.shape(position_sampling_rate_Hz))) # np.shape(position_sampling_rate_Hz): (59308,)
print('type(position_sampling_rate_Hz): {}'.format(type(position_sampling_rate_Hz))) # np.shape(position_sampling_rate_Hz): (59308,)
microseconds_to_seconds_conversion_factor = position_mat_file['microseconds_to_seconds_conversion_factor'].item()
print('type(microseconds_to_seconds_conversion_factor): {}'.format(type(microseconds_to_seconds_conversion_factor))) # np.shape(microseconds_to_seconds_conversion_factor): (59308,)
# print('np.shape(microseconds_to_seconds_conversion_factor): {}'.format(np.shape(microseconds_to_seconds_conversion_factor))) # np.shape(microseconds_to_seconds_conversion_factor): (59308,)

t_rel = t - t[0] # relative to start of position file timestamps
num_samples = len(t)

# active_t_start = t[0] # absolute t_start
active_t_start = 0.0 # relative t_start
# active_t_start = (spikes_df.t.loc[spikes_df.x.first_valid_index()] * timestamp_scale_factor) # actual start time in seconds
temp_position = core.Position(traces=np.vstack((x, y)), computed_traces=np.full([1, num_samples], np.nan), t_start=active_t_start, sampling_rate=position_sampling_rate_Hz)


Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position_info.mat...
done.
np.shape(t): (59308,)
np.shape(x): (59308,)
np.shape(y): (59308,)
type(position_sampling_rate_Hz): <class 'float'>
type(microseconds_to_seconds_conversion_factor): <class 'float'>


In [9]:
sess.epochs.to_dataframe()

,start,stop,label,duration
0,506917.636049,508656.789413,maze1,1739.153364
1,508656.789413,508850.056054,maze2,193.266641


In [10]:
np.shape(sess.position.traces) # (2, 1014937)
print('np.shape(sess.position.traces): {}'.format(np.shape(sess.position.traces))) # (2, 59308)
print('sess.position.time: {}\n sess.position.duration: {}\n sess.position.time[-1]: {}\n sess.position.time[0]: {}\n sess.position.sampling_rate: {}\n sess.position.t_start: {}\n sess.position.t_stop: {}\n'.format(
    sess.position.time,
    sess.position.duration,
    sess.position.time[-1],
    sess.position.time[0],
    sess.position.sampling_rate,
    sess.position.t_start,
    sess.position.t_stop))

# sess.position.time
# sess.position.to_dict()
sess.position.time # [603785.449121, ..., 605893.97363495]
# d = {"t": sess.position.time.flatten(), "x": sess.position.x.flatten(), "y": sess.position.y.flatten(), "lin_pos": sess.position.linear_pos.flatten()}
df = pd.DataFrame({"t": sess.position.time.flatten(), "x": sess.position.x.flatten(), "y": sess.position.y.flatten(), "lin_pos": sess.position.linear_pos.flatten()})
df
# 42305.
# sess.position.to_dataframe()

np.shape(sess.position.traces): (2, 59308)
sess.position.time: [0.00000000e+00 3.33675232e-02 6.67350464e-02 ... 1.97886096e+03
 1.97889433e+03 1.97892770e+03]
 sess.position.duration: 1978.9276969605419
 sess.position.time[-1]: 1978.9276969605419
 sess.position.time[0]: 0.0
 sess.position.sampling_rate: 29.96976599553983
 sess.position.t_start: 0.0
 sess.position.t_stop: 1978.9276969605419



,t,x,y,lin_pos
0,0.000000,104.287278,100.981569,NaN
1,0.033368,104.287278,100.981569,NaN
2,0.066735,104.287278,100.981569,NaN
3,0.100103,104.287278,100.981569,NaN
4,0.133470,104.287278,100.981569,NaN
...,...,...,...,...
59303,1978.794227,75.767781,140.059733,NaN
59304,1978.827594,75.766816,140.106978,NaN
59305,1978.860962,75.735555,140.106496,NaN
59306,1978.894329,75.593275,140.042007,NaN


In [12]:
# sess.epochs['maze1'] # array([506917.636049  , 508656.78941312])
active_epochLabelName = 'maze1'
active_epoch_times = sess.epochs[active_epochLabelName] # array([11070, 13970], dtype=int64)
print('active_epoch_times: {}'.format(active_epoch_times))
acitve_epoch_timeslice_indicies = sess.position.time_slice_indicies(active_epoch_times[0], active_epoch_times[1])
print('active_epoch_times[0]: {}, active_epoch_times[1]: {}'.format(active_epoch_times[0], active_epoch_times[1]))
print('acitve_epoch_timeslice_indicies: {}'.format(acitve_epoch_timeslice_indicies))
active_epoch_pos = sess.position.time_slice(active_epoch_times[0], active_epoch_times[1])
print('active_epoch_pos: {}'.format(active_epoch_pos))        
# sess.neurons.t_start # 0
# sess.neurons.t_stop # 68624338; 68,624,338

active_epoch_times: [506917.636049   508656.78941312]
active_epoch_times[0]: 506917.63604899996, active_epoch_times[1]: 508656.7894131185
acitve_epoch_timeslice_indicies: [False False False ... False False False]
active_epoch_pos: <neuropy.core.position.Position object at 0x0000019934590D00>


In [ ]:
from sklearn.decomposition importepochsCA
from sklearn.manifold import Isomap
from scipy.ndimage import gaussian_filter1d

def linearize_position(position: core.Position, sample_sec=3, method="isomap", sigma=2):
    """linearize trajectory. Use method='PCA' for off-angle linear track, method='ISOMAP' for any non-linear track.
    ISOMAP is more versatile but also more computationally expensive.

    Parameters
    ----------
    track_names: list of track names, each must match an epoch in epochs class.
    sample_sec : int, optional
        sample a point every sample_sec seconds for training ISOMAP, by default 3. Lower it if inaccurate results
    method : str, optional
        by default 'ISOMAP' (for any continuous track, untested on t-maze as of 12/22/2020) or
        'PCA' (for straight tracks)

    """
    xpos = position.x
    ypos = position.y

    xy_pos = np.vstack((xpos, ypos)).T
    xlinear = None
    if method.lower() == "pca":
        pca = PCA(n_components=1)
        xlinear = pca.fit_transform(xy_pos).squeeze()
    elif method.lower() == "isomap":
        imap = Isomap(n_neighbors=5, n_components=2)
        # downsample points to reduce memory load and time
        pos_ds = xy_pos[0 : -1 : np.round(int(position.sampling_rate) * sample_sec)]
        imap.fit(pos_ds)
        iso_pos = imap.transform(xy_pos)
        # Keep iso_pos here in case we want to use 2nd dimension (transverse to track) in future...
        if iso_pos.std(axis=0)[0] < iso_pos.std(axis=0)[1]:
            iso_pos[:, [0, 1]] = iso_pos[:, [1, 0]]
        xlinear = iso_pos[:, 0]
    else:
        print('ERROR: invalid method name: {}'.format(method))
    xlinear = gaussian_filter1d(xlinear, sigma=sigma)
    return core.Position(
        traces=xlinear, t_start=position.t_start, sampling_rate=position.sampling_rate
    )

def compute_linearized_position(session, epochLabelName='maze1', method='isomap'):
        # returns Position objects for active_epoch_pos and linear_pos
        active_epoch_times = session.epochs[epochLabelName] # array([11070, 13970], dtype=int64)
        acitve_epoch_timeslice_indicies = session.position.time_slice_indicies(active_epoch_times[0], active_epoch_times[1])
        active_epoch_pos = session.position.time_slice(active_epoch_times[0], active_epoch_times[1])
        linear_pos = linearize_position(active_epoch_pos, method=method)
        return acitve_epoch_timeslice_indicies, active_epoch_pos, linear_pos

print('computing linear positions for all active epochs for session...')
# end result will be session.computed_traces of the same length as session.traces in terms of frames, with all non-maze times holding NaN values
sess.position.computed_traces = np.full([1, sess.position.traces.shape[1]], np.nan)
# acitve_epoch_timeslice_indicies1, active_positions_maze1, linearized_positions_maze1 = DataSession.compute_linearized_position(session, epochLabelName='maze', method='pca')
# session.position.computed_traces[0,  acitve_epoch_timeslice_indicies1] = linearized_positions_maze1.traces
acitve_epoch_timeslice_indicies1, active_positions_maze1, linearized_positions_maze1 = compute_linearized_position(sess, epochLabelName='maze1', method='pca')
acitve_epoch_timeslice_indicies2, active_positions_maze2, linearized_positions_maze2 = compute_linearized_position(sess, epochLabelName='maze2', method='pca')
sess.position.computed_traces[0,  acitve_epoch_timeslice_indicies1] = linearized_positions_maze1.traces
sess.position.computed_traces[0,  acitve_epoch_timeslice_indicies2] = linearized_positions_maze2.traces
print('done.')

In [ ]:
## NOTE: The only place that I need to be careful with indexing is with sess.position properties, as these appear to be represented in terms of the number of 60Hz samples instead of in seconds like the Neurons and other classes.
# active_epoch_name = 'maze1'
active_epoch_name = 'maze'
active_subplots_shape = (1,1) # Single subplot
# active_subplots_shape = '1|2' # 1 subplot on left, two on right
active_config = InteractivePlaceCellConfig(active_epoch_name,
                    VideoOutputModeConfig(active_frame_range=np.arange(11070.0, 13970.0), video_output_parent_dir=Path('output', active_epoch_name), active_is_video_output_mode=False),
                    PlottingConfig(output_subplots_shape=active_subplots_shape, output_parent_dir=Path('output', active_epoch_name))) # '3|1
active_epoch_times = sess.epochs[active_config.active_epochs] 
# active_epoch_times = sess.epochs['maze2']  # np.arange(15200.0, 18000.0)
# active_epoch_times = sess.epochs['maze1']  # array([11070, 13970], dtype=int64)
print('Constraining to epoch with times (start: {}, end: {})'.format(active_epoch_times[0], active_epoch_times[1]))

# (start: 603785449121.0, end: 603785451229.5245); (start: 603,785,449,121.0, end: 603,785,451,229.5245)
# active_epoch_session_Neurons = sess.neurons.get_neuron_type('pyramidal')
# active_epoch_session_Neurons = sess.neurons.time_slice(active_epoch_times[0], active_epoch_times[1]) # Filter by pyramidal cells only, returns a core.Neurons object with its spiketrains filtered for the provided start/end times
active_epoch_session_Neurons = sess.neurons.get_neuron_type('pyramidal').time_slice(active_epoch_times[0], active_epoch_times[1]) # Filter by pyramidal cells only, returns a core.Neurons object with its spiketrains filtered for the provided start/end times
active_epoch_position_times_index_mask = sess.position.time_slice_indicies(active_epoch_times[0], active_epoch_times[1]) # a Boolean selection mask
active_epoch_position_times = sess.position.time[active_epoch_position_times_index_mask] # The actual times
active_epoch_relative_position_times = active_epoch_position_times - active_epoch_position_times[0] # Subtract off the first index, so that it becomes zero
active_epoch_pos = sess.position.time_slice(active_epoch_times[0], active_epoch_times[1]) # active_epoch_pos's .time and start/end are all valid
# have active_epoch_position_times: the actual times each position sample occured in seconds, active_epoch_relative_position_times: the same as active_epoch_position_times but starting at zero. Finally, have a complete active_epoch_pos object

print(active_epoch_pos)
str(active_epoch_pos.linear_pos_obj)

active_epoch_session_Neurons.n_spikes

In [ ]:
## Compute Placefields if needed:
from neuropy.analyses import Pf1D, Pf2D
from neuropy.plotting.spikes import get_neuron_colors

should_force_recompute_placefields = True

try: active_epoch_placefields
except NameError: active_epoch_placefields = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
if ((active_epoch_placefields is None) or should_force_recompute_placefields):
    print('Recomputing active_epoch_placefields...')
    active_epoch_placefields1D = Pf1D(neurons=active_epoch_session_Neurons, position=active_epoch_pos.linear_pos_obj, speed_thresh=4, grid_bin=4)
    ax_pf_1D = active_epoch_placefields1D.plot_ratemaps()
    active_pf_1D_identifier_string = '1D Placefields - {}'.format(active_epoch_name)
    plt.title(active_pf_1D_identifier_string)
    active_pf_1D_output_filename = '{}.pdf'.format(active_pf_1D_identifier_string)
    active_pf_1D_output_filepath = active_config.plotting_config.active_output_parent_dir.joinpath(active_pf_1D_output_filename)
    print('Saving 1D Placefield image out to "{}"...'.format(active_pf_1D_output_filepath))
    plt.savefig(active_pf_1D_output_filepath)          
    # active_epoch_placefields = active_epoch_placefields1D
    print('done.')
else:
    print('active_epoch_placefields already exists, reusing it')
    
try: active_epoch_placefields2D
except NameError: active_epoch_placefields2D = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
if ((active_epoch_placefields2D is None) or should_force_recompute_placefields):
    print('Recomputing active_epoch_placefields2D...')
    active_epoch_placefields2D = Pf2D(neurons=active_epoch_session_Neurons, position=active_epoch_pos, speed_thresh=4, grid_bin=4)
    # active_epoch_placefields2D.plotMap(subplots=(1, 1),figsize=(10,10))
    # active_pf_2D_figures, active_pf_2D_gs = active_epoch_placefields2D.plotMap(subplots=(4, 4),figsize=(10, 10))
    # active_epoch_placefields2D.occupancy # (37, 38) = (len(active_epoch_placefields2D.ratemap.xbin_centers), len(active_epoch_placefields2D.ratemap.ybin_centers))
    active_pf_occupancy_2D_identifier_string = '2D Occupancy - {}'.format(active_epoch_name)
    occupancy_fig = plt.figure()
    occupancy_ax = occupancy_fig.gca()
    im = occupancy_ax.pcolorfast(
        active_epoch_placefields2D.ratemap.xbin_centers,
        active_epoch_placefields2D.ratemap.ybin_centers,
        np.rot90(np.fliplr(active_epoch_placefields2D.occupancy)) / np.max(active_epoch_placefields2D.occupancy),
        cmap="jet",
        vmin=0,
    )  # rot90(flipud... is necessary to match plotRaw configuration.
    plt.title(active_pf_occupancy_2D_identifier_string)
    plt.show()
    # Save ocupancy figure out to disk:
    active_pf_occupancy_2D_output_filename = '{}.pdf'.format(active_pf_occupancy_2D_identifier_string)
    active_pf_occupancy_2D_output_filepath = active_config.plotting_config.active_output_parent_dir.joinpath(active_pf_occupancy_2D_output_filename)
    print('Saving 2D Placefield image out to "{}"...'.format(active_pf_occupancy_2D_output_filepath))
    occupancy_fig.savefig(active_pf_occupancy_2D_output_filepath)
    print('done.')
    ## 2D Tuning Curves Figure:
    active_pf_2D_figures, active_pf_2D_gs = active_epoch_placefields2D.plotMap(subplots=(7, 7),figsize=(10, 10))
    # active_epoch_placefields2D.plotRaw()
    active_pf_2D_identifier_string = '2D Placefields - {}'.format(active_epoch_name)
    # plt.title(active_pf_2D_identifier_string)
    active_pf_2D_output_filename = '{}.pdf'.format(active_pf_2D_identifier_string)
    active_pf_2D_output_filepath = active_config.plotting_config.active_output_parent_dir.joinpath(active_pf_2D_output_filename)
    print('Saving 2D Placefield image out to "{}"...'.format(active_pf_2D_output_filepath))
    for aFig in active_pf_2D_figures:
        aFig.savefig(active_pf_2D_output_filepath)
    print('done.')
else:
    print('active_epoch_placefields already exists, reusing it')
    
active_epoch_placefields = active_epoch_placefields2D

# Get the cell IDs that have a good place field mapping:
good_placefield_neuronIDs = np.array(active_epoch_placefields.ratemap.neuron_ids) # in order of ascending ID
good_placefield_neuronIDs

In [ ]:
## Plots the tuning curves one at a time, and also can plot them without very low probabilitiy entries (by setting these entries to np.nan)
curr_tuning_curves = active_epoch_placefields.ratemap.tuning_curves
# curr_tuning_curves = active_epoch_placefields.ratemap.normalized_tuning_curves
curr_index = 5
curr_unitID = active_epoch_placefields2D.cell_ids[curr_index]
curr_identifier_string = '2D Placemap - {} - Unit[{}] (ID {})'.format(active_epoch_name, curr_index, curr_unitID)
curr_data = curr_tuning_curves[curr_index]
curr_data_min_val = np.min(curr_data)
curr_data_max_val = np.max(curr_data)
curr_data[curr_data < 0.01] = np.nan
print('curr_data_min_val: {}; curr_data_max_val: {}'.format(curr_data_min_val, curr_data_max_val))
curr_fig = plt.figure()
curr_ax = curr_fig.gca()
im = curr_ax.pcolorfast(
    active_epoch_placefields2D.ratemap.xbin_centers,
    active_epoch_placefields2D.ratemap.ybin_centers,
    np.rot90(np.fliplr(curr_data)),
    cmap="jet",
    vmin=0,
)  # rot90(flipud... is necessary to match plotRaw configuration.
plt.title(curr_identifier_string)
plt.show()

In [ ]:
# pf_ax, pf_sort_ind, pf_colors = active_epoch_placefields.plot_ratemaps()
# from neuropy.utils import mathutil
# curr_tuning_curves = mathutil.min_max_scaler(active_epoch_placefields.ratemap.tuning_curves) # 37x25x27 ndarray
# curr_tuning_curves = active_epoch_placefields.ratemap.tuning_curves
curr_tuning_curves = active_epoch_placefields.ratemap.normalized_tuning_curves
# sort_ind = np.argsort(np.argmax(curr_tuning_curves, axis=1))
# ind = np.unravel_index(np.argsort(curr_tuning_curves, axis=None), curr_tuning_curves.shape)
num_curr_tuning_curves = len(curr_tuning_curves)
pf_sort_ind = np.arange(num_curr_tuning_curves)
pf_colors = get_neuron_colors(pf_sort_ind)
pf_sort_ind = np.array([int(pf_sort_ind[i]) for i in np.arange(len(pf_sort_ind))]) # convert to integer scalar array
pf_sorted_good_placefield_neuronIDs = good_placefield_neuronIDs[pf_sort_ind]
reverse_color_sort_indices = np.argsort(pf_sort_ind)
pf_colors = pf_colors[:, reverse_color_sort_indices] # pf_colors shape is still (4, 31)
active_epoch_session_Neurons = active_epoch_session_Neurons.get_by_id(good_placefield_neuronIDs) # Filter by good placefields only

# Unpacking final values into separate variables:
# Spike variables:
num_cells = active_epoch_session_Neurons.n_neurons
spike_list = active_epoch_session_Neurons.spiketrains
cell_ids = active_epoch_session_Neurons.neuron_ids
flattened_spikes = active_epoch_session_Neurons.get_flattened_spikes() # get_flattened_spikes(..) returns a FlattenedSpiketrains object
# Position variables: t, x, y
t = active_epoch_pos.time
x = active_epoch_pos.x
y = active_epoch_pos.y
linear_pos = active_epoch_pos.linear_pos
# speeds = active_epoch_pos.speed # note this has 1 less element than active_epoch_pos.x
# Determine the x and y positions each spike occured for each cell
spike_positions_list = build_spike_positions_list(spike_list, t, x, y)
reverse_cellID_idx_lookup_map = build_cellID_reverse_lookup_map(cell_ids)
print('num_cells: {}'.format(num_cells))
print('cell_ids: {}'.format(cell_ids)) # cell_ids is now a regular python list with 57 elements

active_cells_colormap = pf_colors.T # Make the colormap from the listed colors
active_cells_listed_colormap = ListedColormap(active_cells_colormap)

# Gets the flattened spikes, sorted in ascending timestamp for all cells.
num_flattened_spikes = np.size(flattened_spikes.flattened_spike_times)
print('num_flattened_spikes: {}'.format(num_flattened_spikes))
# Build the Active UnitIDs
flattened_spike_active_unitIdentities = np.array([int(reverse_cellID_idx_lookup_map[original_cellID]) for original_cellID in flattened_spikes.flattened_spike_identities]) # since flattened_spikes.flattened_spike_identities is already sorted, don't double sort
## Build the flattened spike positions list
flattened_spike_positions_list = np.concatenate(tuple(spike_positions_list), axis=1) # needs tuple(...) to conver the list into a tuple, which is the format it expects
flattened_spike_positions_list = flattened_spike_positions_list[:, flattened_spikes.flattened_sort_indicies] # ensure the positions are ordered the same as the other flattened items so they line up
print('flattened_spike_positions_list: {}'.format(np.shape(flattened_spike_positions_list)))

In [ ]:
# have active_epoch_position_times: the actual times each position sample occured in seconds, active_epoch_relative_position_times: the same as active_epoch_position_times but starting at zero
# describe the movement

from PhoPositionalData.plotting.visualization_window import VisualizationWindow # Used to build "Windows" into the data points such as the window defining the fixed time period preceeding the current time where spikes had recently fired, etc.

# Split the position data into equal sized chunks to be displayed at a single time. These will look like portions of the trajectory and be used to animate. # Chunk the data to create the animation.
curr_plot_update_step = 1 # Update every frame
curr_plot_update_frequency = curr_plot_update_step * active_epoch_pos.sampling_rate # number of updates per second (Hz)
num_time_points = active_epoch_pos.n_frames / curr_plot_update_step
print('active_epoch_pos.sampling_rate (Hz): {}'.format(active_epoch_pos.sampling_rate))

# curr_window_duration = 2.5 # in seconds
# curr_view_window_length_samples = int(np.floor(curr_window_duration * active_epoch_pos.sampling_rate)) # number of samples the window should last
# recent_spikes_window = VisualizationWindow(duration_seconds=curr_window_duration, duration_num_frames=curr_view_window_length_samples)

# curr_recently_window_duration = 0.5 # in seconds
# curr_view_window_length_samples = int(np.floor(curr_window_duration * active_epoch_pos.sampling_rate)) # number of samples the window should last

## Simplified with just two windows:
longer_spikes_window = VisualizationWindow(duration_seconds=1024.0, sampling_rate=active_epoch_pos.sampling_rate) # have it start clearing spikes more than 30 seconds old
curr_view_window_length_samples = longer_spikes_window.duration_num_frames # number of samples the window should last
print('longer_spikes_window - curr_view_window_length_samples - {}'.format(curr_view_window_length_samples))

recent_spikes_window = VisualizationWindow(duration_seconds=1.0, sampling_rate=active_epoch_pos.sampling_rate)
curr_view_window_length_samples = recent_spikes_window.duration_num_frames # number of samples the window should last
print('recent_spikes_window - curr_view_window_length_samples - {}'.format(curr_view_window_length_samples))

## Build the sliding windows:
from numpy.lib.stride_tricks import sliding_window_view
# build a sliding window to be able to retreive the correct flattened indicies for any given timestep
active_epoch_position_linear_indicies = np.arange(np.size(active_epoch_position_times))
pre_computed_window_sample_indicies = recent_spikes_window.build_sliding_windows(active_epoch_position_linear_indicies)
# print('pre_computed_window_sample_indicies: {}\n shape: {}'.format(pre_computed_window_sample_indicies, np.shape(pre_computed_window_sample_indicies)))

## New Pre Computed Indicies Way:
z_fixed = np.full((recent_spikes_window.duration_num_frames,), 1.1)

In [ ]:
## ICA and PCA Analysis
should_show_2D_ICA_plots = False
from PhoPositionalData.analysis.neuronal_dimensionality_reduction import runAnalysis_PCAandICA
active_session_ensembles, template, zsc_template, pca_data = runAnalysis_PCAandICA(active_epoch_session_Neurons, bin_size=0.250, frate_thresh=0.1, should_plot=should_show_2D_ICA_plots, active_cells_colormap=active_cells_colormap)

## Main Spike/Placemap plotting:

In [ ]:
from PhoPositionalData.plotting.gui import customize_default_pyvista_theme, print_controls_helper_text
customize_default_pyvista_theme() # Sets the default theme values to those specified in my imported file
# This defines the position of the vertical/horizontal splitting, in this case 40% of the vertical/horizontal dimension of the window
# pv.global_theme.multi_rendering_splitting_position = 0.40
pv.global_theme.multi_rendering_splitting_position = 0.80

from PhoPositionalData.plotting.spikeAndPositions import build_active_spikes_plot_data, build_flat_map_plot_data, build_spike_spawn_effect_light_actor, spike_geom_circle, spike_geom_box, spike_geom_cone, animal_location_circle, animal_location_trail_circle
from PhoPositionalData.plotting.spikeAndPositions import InteractiveSliderWrapper # for wrapping the slider
num_time_points = active_epoch_pos.n_frames / curr_plot_update_step
# print('num_time_points: {}\n'.format(num_time_points))

## Opacity Helpers:
last_only_opacity_values = np.zeros([curr_view_window_length_samples,])
last_only_opacity_values[-1] = 1.0
# gradually_fading_opacity_values = np.arange(curr_view_window_length_samples)
gradually_fading_opacity_values = np.linspace(0.0, 1.0, curr_view_window_length_samples)
long_gradually_fading_opacity_values = np.linspace(0.0, 1.0, longer_spikes_window.duration_num_frames)
sharply_fading_opacity_values = np.linspace(0.0, 0.6, curr_view_window_length_samples)
# sharply_fading_opacity_values[-1] = 0.1 # last element (corresponding to current position) is set to 1.0

# active_trail_opacity_values = last_only_opacity_values.copy()
# active_trail_opacity_values = gradually_fading_opacity_values.copy()
active_trail_opacity_values = sharply_fading_opacity_values.copy()
# print('active_trail_opacity_values: {}\n'.format(np.shape(active_trail_opacity_values)))
# active_trail_size_values = np.full([curr_view_window_length_samples,], 0.6) # all have a scale of 0.6
active_trail_size_values = np.linspace(0.2, 0.6, curr_view_window_length_samples) # fade from a scale of 0.2 to 0.6
# active_trail_size_values[-1] = 6.0 # except for the end (current) point, which has a scale of 1.0
# active_trail_size_values = sharply_fading_opacity_values.copy()

In [ ]:
from PhoPositionalData.plotting.spikeAndPositions import plot_placefields2D, update_plotVisiblePlacefields2D
pActiveTuningCurvesPlotter = pvqt.BackgroundPlotter(window_size=(1920, 1080), shape=(1,1), off_screen=False) # Use just like you would a pv.Plotter() instance
pActiveTuningCurvesPlotter.clear()
# Plot the flat arena
pdata_maze, pc_maze = build_flat_map_plot_data(x, y)
pActiveTuningCurvesPlotter.add_mesh(pc_maze, name='maze_bg', label='maze', color="black", render=True)
pActiveTuningCurvesPlotter, tuningCurvePlotActors, tuningCurvePlotLegendActor = plot_placefields2D(pActiveTuningCurvesPlotter, active_epoch_placefields, pf_colors, zScalingFactor=10.0)    

# tuningCurvePlotActors[1].VisibilityOn()
# legendActor = pTuningCurves.add_legend(name='tuningCurvesLegend', origin=[0.9, 0.0], size=[0.1, 1.0]) 

In [ ]:
# temp_legendNumberOfEntries = tuningCurvePlotLegendActor.GetNumberOfEntries()
# print('tuningCurvePlotLegendActor.GetPosition(): {}; tuningCurvePlotLegendActor.GetPosition2(): {};'.format(tuningCurvePlotLegendActor.GetPosition(), tuningCurvePlotLegendActor.GetPosition2()))
# # print('tuningCurvePlotLegendActor.GetPositionCoordinate(): {}; tuningCurvePlotLegendActor.GetPosition2Coordinate(): {}'.format(tuningCurvePlotLegendActor.GetPositionCoordinate(), tuningCurvePlotLegendActor.GetPosition2Coordinate()))
# print('tuningCurvePlotLegendActor.GetHeight(): {}; tuningCurvePlotLegendActor.GetWidth(): {};'.format(tuningCurvePlotLegendActor.GetHeight(), tuningCurvePlotLegendActor.GetWidth()))
# print('temp_legendNumberOfEntries: {}'.format(temp_legendNumberOfEntries))
# # tuningCurvePlotLegendActor.GetPropertyKeys()
# for i in np.arange(temp_legendNumberOfEntries):    
#     temp_legendEntryString = tuningCurvePlotLegendActor.GetEntryString(i)
#     temp_legendEntrySymbol = tuningCurvePlotLegendActor.GetEntrySymbol(i)
    
#     # temp_legendActors = tuningCurvePlotLegendActor.GetActors(i) 

In [ ]:
## Test Interactively Adjusting Visibility:
curr_is_visible = np.full([len(tuningCurvePlotActors), 1], False)
curr_is_visible[12] = True
update_plotVisiblePlacefields2D(tuningCurvePlotActors, curr_is_visible)
# pActiveTuningCurvesPlotter.disable_depth_peeling()
pActiveTuningCurvesPlotter.update()

In [ ]:
## Slider with Callback Function Example:

######################
# General Plotting Method:    
def on_slider_update_mesh(value):
    curr_i = int(value)    
    active_window_sample_indicies = np.squeeze(pre_computed_window_sample_indicies[curr_i,:]) # Get the current precomputed indicies for this curr_i
    
    ## Spike Plotting:
    # Get the times that fall within the current plot window:
    curr_time_fixedSegments = t[active_window_sample_indicies] # New Way
    t_start = curr_time_fixedSegments[0]
    t_stop = curr_time_fixedSegments[-1]
    # print('Constraining to curr_time_fixedSegments with times (start: {}, end: {})'.format(t_start, t_stop))
    # print('curr_time_fixedSegments: {}'.format(curr_time_fixedSegments))
    curr_text_rendering_string = 'curr_i: {:d}; (t_start: {:.2f}, t_stop: {:.2f})'.format(curr_i, t_start, t_stop) # :.3f
    p.add_text(curr_text_rendering_string, name='lblCurrent_spike_range', position='lower_right', color='white', shadow=True, font_size=10)

    ## Historical Spikes:
    # active_included_all_historical_indicies = (flattened_spikes.flattened_spike_times < t_stop) # Accumulate Spikes mode. All spikes occuring prior to the end of the frame (meaning the current time) are plotted
    historical_t_start = (t_stop - longer_spikes_window.duration_seconds) # Get the earliest time that will be included in the search
    active_included_all_historical_indicies = ((flattened_spikes.flattened_spike_times > historical_t_start) & (flattened_spikes.flattened_spike_times < t_stop)) # Two Sided Range Mode
    historical_spikes_pdata, historical_spikes_pc = build_active_spikes_plot_data(flattened_spikes.flattened_spike_times[active_included_all_historical_indicies],
                                                                                  flattened_spike_active_unitIdentities[active_included_all_historical_indicies],
                                                                                  flattened_spike_positions_list[:, active_included_all_historical_indicies],
                                                                                  spike_geom=spike_geom_box.copy())
    if historical_spikes_pc.n_points >= 1:
        historical_main_spikes_mesh = p.add_mesh(historical_spikes_pc, name='historical_spikes_main', scalars='cellID', cmap=active_cells_listed_colormap, show_scalar_bar=False, lighting=True, render=False)

    ## Actively Firing Spikes:
    recent_spikes_t_start = (t_stop - recent_spikes_window.duration_seconds) # Get the earliest time that will be included in the recent spikes
    # print('recent_spikes_t_start: {}; t_start: {}'.format(recent_spikes_t_start, t_start))
    active_included_recent_only_indicies = ((flattened_spikes.flattened_spike_times > recent_spikes_t_start) & (flattened_spikes.flattened_spike_times < t_stop)) # Two Sided Range Mode
    # active_included_recent_only_indicies = ((flattened_spikes.flattened_spike_times > t_start) & (flattened_spikes.flattened_spike_times < t_stop)) # Two Sided Range Mode
    recent_only_spikes_pdata, recent_only_spikes_pc = build_active_spikes_plot_data(flattened_spikes.flattened_spike_times[active_included_recent_only_indicies],
                                                                                    flattened_spike_active_unitIdentities[active_included_recent_only_indicies],
                                                                                    flattened_spike_positions_list[:, active_included_recent_only_indicies],
                                                                                    spike_geom=spike_geom_cone.copy())
    if recent_only_spikes_pc.n_points >= 1:
        recent_only_main_spikes_mesh = p.add_mesh(recent_only_spikes_pc, name='recent_only_spikes_main', scalars='cellID', cmap=active_cells_listed_colormap, show_scalar_bar=False, lighting=False, render=False) # color='white'
        
    ## Animal Position and Location Trail Plotting:
    point_cloud_fixedSegements_positionTrail = np.column_stack((x[active_window_sample_indicies], y[active_window_sample_indicies], z_fixed))
    pdata_positionTrail = pv.PolyData(point_cloud_fixedSegements_positionTrail.copy()) # a mesh
    pdata_positionTrail.point_data['pho_fade_values'] = active_trail_opacity_values
    pdata_positionTrail.point_data['pho_size_values'] = active_trail_size_values
    # create many spheres from the point cloud
    pc_positionTrail = pdata_positionTrail.glyph(scale='pho_size_values', geom=animal_location_trail_circle)
    animal_location_trail_mesh = p.add_mesh(pc_positionTrail, name='animal_location_trail', ambient=0.6, opacity='linear_r', scalars='pho_fade_values', nan_opacity=0.0,
                                            show_edges=False, render_lines_as_tubes=True, show_scalar_bar=False, use_transparency=True, render=False) # works to render a heat colored (most recent==hotter) position

    ## Animal Current Position:
    curr_animal_point = point_cloud_fixedSegements_positionTrail[-1,:].copy() # Get the last point
    pdata_current_point = pv.PolyData(curr_animal_point) # a mesh
    pc_current_point = pdata_current_point.glyph(scale=False, geom=animal_location_circle)
    animal_current_location_point_mesh = p.add_mesh(pc_current_point, name='animal_location', color='green', ambient=0.6, opacity=0.5,
                                                    show_edges=True, edge_color=[0.05, 0.8, 0.08], line_width=3.0, nan_opacity=0.0, render_lines_as_tubes=True,
                                                    show_scalar_bar=False, use_transparency=True, render=False) # works to render a heat colored (most recent==hotter) position
    
    p.render() # renders to ensure it's updated after changing the ScalarVisibility above
    # p.update()
    # p.app.processEvents() # not needed probably
    return


################################################
### Build Appropriate Plotter and set it up:
#####################
# Only Create a new BackgroundPlotter if it's needed:
if (active_config.video_output_config.active_is_video_output_mode):
    ## Video mode should use a regular plotter object
    p = pv.Plotter(notebook=False, shape=active_config.plotting_config.subplots_shape, window_size=([1280, 720]), off_screen=True) # , line_smoothing=True, polygon_smoothing=True, multi_samples=8
else:
    try: p
    except NameError: p = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
    if (p is not None):
        if isinstance(p, pvqt.BackgroundPlotter):
            if p.app_window.isHidden():
                print('No open BackgroundPlotter')
                p.close() # Close it to start over fresh
                p = None
                needs_create_new_backgroundPlotter = True
            else:
                print('BackgroundPlotter already open, reusing it.. NOT Forcing creation of a new one!')
                # p.app_window.window().show()
                # p.clear()
                # needs_create_new_backgroundPlotter = False                
                p.close() # Close it to start over fresh
                p = None
                needs_create_new_backgroundPlotter = True
                
        else:
            print('No open BackgroundPlotter, p is a Plotter object')
            p.close()
            p = None
            needs_create_new_backgroundPlotter = True
    else:
        print('No extant BackgroundPlotter')
        needs_create_new_backgroundPlotter = True
    if needs_create_new_backgroundPlotter:
        print('Creating a new BackgroundPlotter')
        p = pvqt.BackgroundPlotter(window_size=(1920, 1080), shape=active_config.plotting_config.subplots_shape, off_screen=False) # Use just like you would a pv.Plotter() instance
        print('done.')

# p.background_color = 'black'

if (not active_config.video_output_config.active_is_video_output_mode):
    #Interactive Mode: Enable interactive controls:
    interactive_timestamp_slider_actor = p.add_slider_widget(on_slider_update_mesh, [0, (num_time_points-1)], title='Trajectory Timestep', event_type='always', style='modern', pointa=(0.025, 0.1), pointb=(0.98, 0.1), fmt='%0.2f') # fmt="%0.2f"
    interactive_timestamp_slider_wrapper = InteractiveSliderWrapper(interactive_timestamp_slider_actor)    
    # interactive_checkbox_actor = p.add_checkbox_button_widget(toggle_animation, value=False, color_on='green')
    helper_controls_text = print_controls_helper_text()
    p.add_text(helper_controls_text, position='upper_left', name='lblControlsHelperText', color='grey', font_size=8.0)
    
    
# Plot the flat arena
pdata_maze, pc_maze = build_flat_map_plot_data(x, y)
p.add_mesh(pc_maze, name='maze_bg', color="black", render=False)
# p.show_grid()
# p.add_axes(line_width=5, labels_off=True)
p.enable_depth_peeling(number_of_peels=4, occlusion_ratio=0) # Supposedly helps with translucency
p.hide_axes()
# p.camera_position = 'xy' # Overhead (top) view
# apply_close_overhead_zoomed_camera_view(p)
apply_close_perspective_camera_view(p)

p.render() # manually render when needed

if active_config.video_output_config.active_is_video_output_mode:
    print('Writing video to {}...'.format(active_config.video_output_config.active_video_output_fullpath))
    p.show(auto_close=False)
    make_mp4_from_plotter(p, active_config.video_output_config.active_frame_range, on_slider_update_mesh, filename=active_config.video_output_config.active_video_output_fullpath, framerate=60) # 60fps
    p.close()
    p = None

# p.show()
                  
print('all done!')

In [ ]:
## Playback animation:

# ## Testing Updating the Timestamp Slider Programmatically:
# interactive_timestamp_slider_wrapper.curr_index
# interactive_timestamp_slider_wrapper.step_index(15)

class InterfaceProperties(object):
    def __init__(self, active_timestamp_slider_wrapper):
        # self.curr_plot_update_step = 1 # Update every frame
        # self.curr_plot_update_frequency = self.curr_plot_update_step * active_epoch_pos.sampling_rate # number of updates per second (Hz)
        # self.num_time_points = active_epoch_pos.n_frames / self.curr_plot_update_step        
        # self.position_trail_max_duration = 0
        # self.max_historical_spikes_age = 0 # How long ago the historical spikes could have been plotted and still not be removed.
        self.active_timestamp_slider_wrapper = active_timestamp_slider_wrapper # Used to actually update the slider when it's appropriate
        self.animation_state = False # Whether it's playing or not

    def __call__(self):
        if self.animation_state:
            # only if animation is currently active:
            self.active_timestamp_slider_wrapper.step_index(15) # TODO: allow variable step size
            pass

        
animate = InterfaceProperties(interactive_timestamp_slider_wrapper)
# define the animation switch
def toggle_animation(state):
    animate.animation_state = state # updates the animation state to the new value

# An unused constant-time callback that calls back every so often to perform updates
p.add_callback(animate, interval=16)  # to be smooth on 60Hz

# A checkbox that decides whether we're playing back at a constant rate or not.
interactive_checkbox_actor = p.add_checkbox_button_widget(toggle_animation, value=False, color_on='green')